12) Obtenga la matriz de distancias euclídeas para todos los contenidos que están en Marte. ¿Cuáles son los dos contenidos que están a menor distancia? (⭐⭐)

**Analisis previo**

1) La matriz pedida sera un RDD cuyos valores definen unívocamente una matriz.

2) No tenemos valores None en las columnas "gt_lat" y "gt_lon" las cuales nos interesan.


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

from google.colab import drive
drive.mount("/content/drive")

import math

In [ ]:
def distancia_euclidea(x,y):
  return math.sqrt((x[0] - y[0])**2 + (x[1] - y[1])**2)

### **Resolucion explicativa**

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/Organizacion de Datos/Colab Notebooks/TP1/geo_tags.csv', header=True, inferSchema=True)
rdd = df.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Pasos

1) Con la transformacion filter() buscamos filtrar los datos para quedarnos solamente con los contenidos de Marte.

2) Con la transformacion map() buscamos obtener registros de la forma (1, (latitud, longitud)) para asi consegir el formato necesario y utilizar la transformacion fullOuterJoin() en el siguiente paso.

In [ ]:
rdd_coordenadas = rdd.filter(lambda x: x.gt_globe == "mars").map(lambda x: (1, (x.gt_lat, x.gt_lon))).cache()

Pasos

3) Con la transformacion fullOuterJoin() buscamos obtener un RDD que resulte como un producto cartesiano.

In [ ]:
rdd_join = rdd_coordenadas.fullOuterJoin(rdd_coordenadas)

Pasos

4) Con la transformacion map() buscamos obtener registros de la forma (x, y, distancia euclidea entre x e y).

5) Con la transformacion filter() buscamos filtrar los datos para quedarnos solamente con los contenidos que sean distintos porque sino su distancia sera 0 (y sera el de menor distancia).

In [ ]:
rdd_matriz = rdd_join.map(lambda x: (x[1][0], x[1][1], distancia_euclidea(x[1][0], x[1][1]))).filter(lambda x: x[0] != x[1])

Pasos

6) Con la accion reduce() buscamos hallar aquel registro que tenga la menor distancia euclidea entre dos contenidos.

In [ ]:
resultado = rdd_matriz.reduce(lambda a,b: a if (a[2] < b[2]) else b)

### **Resolucion reducida**

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/Organizacion de Datos/Colab Notebooks/TP1/geo_tags.csv', header=True, inferSchema=True)
rdd = df.rdd

rdd_coordenadas = rdd.filter(lambda x: x.gt_globe == "mars").map(lambda x: (1, (x.gt_lat, x.gt_lon))).cache()

rdd_join = rdd_coordenadas.fullOuterJoin(rdd_coordenadas)

rdd_matriz = rdd_join.map(lambda x: (x[1][0], x[1][1], distancia_euclidea(x[1][0], x[1][1]))).filter(lambda x: x[0] != x[1])

resultado = rdd_matriz.reduce(lambda a,b: a if (a[2] < b[2]) else b)

### **Respuesta**

In [ ]:
rdd_matriz.take(5)

[((16.9, 141.7), (37.4, 126.4), 25.580070367377793),
 ((16.9, 141.7), (-17.7, 199.9), 67.70819743576106),
 ((16.9, 141.7), (16.3, 125.9), 15.81138830084188),
 ((16.9, 141.7), (-28.7, 226.9), 96.63539724138356),
 ((16.9, 141.7), (7.7, 166.0), 25.983263844251756)]

In [ ]:
resultado

((-2.11722222, 354.47710278),
 (-2.11690278, 354.48254167),
 0.005448262690585825)